In [1]:
import numpy as np
import xarray as xr
import cf_xarray as cfxr
import pandas as pd
import regionmask

In [2]:

fN = ("Datos\Land_and_Ocean_EqualArea.nc") 

ds = xr.open_dataset(fN)
years_approx = np.floor(ds['time'])
temp = ds['temperature'].groupby(years_approx).mean('time')
ds

<xarray.Dataset> Size: 135MB
Dimensions:      (map_points: 15984, time: 2089, month_number: 12)
Coordinates:
    longitude    (map_points) float32 64kB ...
    latitude     (map_points) float32 64kB ...
  * time         (time) float64 17kB 1.85e+03 1.85e+03 ... 2.024e+03 2.024e+03
Dimensions without coordinates: map_points, month_number
Data variables:
    land_mask    (map_points) float64 128kB ...
    temperature  (time, map_points) float32 134MB ...
    climatology  (month_number, map_points) float32 767kB ...
Attributes:
    Conventions:           Berkeley Earth Internal Convention (based on CF-1.5)
    title:                 Native Format Berkeley Earth Surface Temperature A...
    history:               20-Feb-2024 09:56:45
    institution:           Berkeley Earth Surface Temperature Project
    land_source_history:   07-Feb-2024 05:19:02
    ocean_source_history:  20-Feb-2024 08:20:00
    comment:               This file contains Berkeley Earth surface temperat...

In [3]:
longitude = ds.variables['longitude'][:]  
latitude = ds.variables['latitude'][:]
time = ds.variables['time'][:]
land_mask = ds.variables['land_mask'][:]
temperature = ds.variables['temperature'][:]
climatology = ds.variables['climatology'][:]

In [4]:
# Ajustar el enfoque para la conversión a DataFrame, teniendo cuidado con la estructura de los datos

# Convertir los datos de temperatura a un DataFrame
temperature_df = ds['temperature'].to_dataframe()

# Dado que 'to_dataframe()' convierte todas las dimensiones a índices en el DataFrame,
# necesitamos resetear el índice y luego pivotar el DataFrame si es necesario
temperature_df_reset = temperature_df.reset_index()
temperature_pivot_df = temperature_df_reset.pivot(index='time', columns='map_points', values='temperature')

# Simplificar los nombres de las columnas después de pivotar
temperature_pivot_df.columns = [f"point_{i+1}" for i in range(len(temperature_pivot_df.columns))]
# Guardar el DataFrame a un archivo CSV
temperature_pivot_df.to_csv("temperatura.csv", index=True)
# Revisar el resultado para asegurarse de que tiene el formato correcto
temperature_pivot_df.head()

,point_1,point_2,point_3,point_4,point_5,point_6,point_7,point_8,point_9,point_10,...,point_15975,point_15976,point_15977,point_15978,point_15979,point_15980,point_15981,point_15982,point_15983,point_15984
time,,,,,,,,,,,,,,,,,,,,,
1850.041667,NaN,NaN,NaN,NaN,NaN,NaN,0.464756,0.681552,0.785687,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1850.125000,NaN,NaN,NaN,NaN,NaN,NaN,-1.490464,-1.236933,-0.806517,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1850.208333,NaN,NaN,NaN,NaN,NaN,NaN,-0.157294,-0.230608,-0.166150,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1850.291667,NaN,NaN,NaN,NaN,NaN,NaN,-0.139167,-0.058191,-0.072071,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1850.375000,NaN,NaN,NaN,NaN,NaN,NaN,-0.878162,-0.964869,-1.003502,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:

# Suponemos que 'ds' es tu xarray.Dataset con datos de temperatura, y que tiene
# variables de 'latitude' y 'longitude' correctamente definidas.

# Define las regiones AR6 utilizando regionmask
regions_ar6 = regionmask.defined_regions.ar6.all

# Crear la máscara para todas las regiones en tus datos
mask = regions_ar6.mask(ds, lon_name='longitude', lat_name='latitude')

# Inicializa un diccionario para almacenar los DataArrays filtrados por región
regions_temperature = {}

for region_id in regions_ar6.numbers:
    # Filtra los datos para la región actual usando la máscara
    regions_temperature[region_id] = ds.where(mask == region_id)

# A este punto, `regions_temperature` contiene un DataArray para cada región, donde
# cada DataArray solo incluye los datos de temperatura para esa región específica.


C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\regionmask\core\mask.py:179: FutureWarning: Passing 'lon_name' and 'lat_name' was deprecated in v0.10.0. Please pass the coordinates directly, e.g., `mask*(ds[lon_name], ds[lat_name])`.
  warnings.warn(


In [6]:
# Inicializa una lista para almacenar las series de pandas
region_series_list = []

# Recorre el diccionario de regiones para construir las series
for region_id, region_ds in regions_temperature.items():
    # Asume que 'temperature' es la variable de interés y que ya hemos calculado el promedio a lo largo del tiempo
    # Aquí, tomamos el promedio a lo largo de todas las dimensiones excepto 'time', lo que debería darte una serie temporal de temperatura promedio
    temp_series = region_ds['temperature'].mean(dim=["map_points"]).to_series()
    temp_series.name = f"Region {region_id}"
    region_series_list.append(temp_series)

# Construir un DataFrame de pandas a partir de las series
temperature_df = pd.concat(region_series_list, axis=1)
# Guardar el DataFrame a un archivo CSV
temperature_df.to_csv("temperaturaregionesipcc.csv", index=True)
temperature_df

,Region 0,Region 1,Region 2,Region 3,Region 4,Region 5,Region 6,Region 7,Region 8,Region 9,...,Region 48,Region 49,Region 50,Region 51,Region 52,Region 53,Region 54,Region 55,Region 56,Region 57
time,,,,,,,,,,,,,,,,,,,,,
1850.041667,0.136235,-1.330789,0.008371,-1.081719,1.557231,1.357871,0.177940,0.005466,-0.013133,-0.725280,...,-0.754152,0.052812,-0.122649,-0.443389,-0.459247,-0.106958,-0.059162,-0.013802,-0.037105,-0.228845
1850.125000,-2.766186,0.903566,0.457267,-1.672187,-0.238511,0.841829,-0.398261,0.037664,-0.005968,-0.345103,...,-0.929530,0.314541,-0.209424,-0.341581,-1.092278,-0.146366,0.671129,0.367911,0.032516,-0.330520
1850.208333,-0.290138,-1.243747,-0.128986,-2.131074,-1.220055,-0.509307,-0.421122,0.153145,-0.047957,-0.186628,...,-0.766979,-0.177736,-0.228850,-0.490354,-0.456664,-0.462185,-0.215787,-0.304528,-0.054666,0.138855
1850.291667,0.248451,-1.428141,-1.840282,-0.240155,-3.188819,-1.529099,-0.744390,-0.017132,-0.094589,-0.157292,...,-0.403917,-0.471322,-0.146334,-0.470825,-0.542606,-0.543796,-0.349228,-0.271987,-0.158646,0.081552
1850.375000,0.256227,-1.246235,-1.437419,-0.391208,-1.781213,-1.727687,-1.144015,0.459868,0.275922,-0.132240,...,0.048388,-0.553649,0.094990,-0.382022,-0.085905,-0.260265,-0.337365,-0.180721,-0.149693,-0.609608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023.708333,1.005331,2.255795,3.185757,1.018099,2.766294,1.684024,1.990303,1.745924,1.494877,2.334448,...,1.752611,0.615157,1.547976,1.210686,0.576096,1.606667,0.921754,0.997689,0.984593,1.339261
2023.791667,1.260326,2.812672,3.510752,0.684375,0.704209,1.683892,1.204790,1.629442,1.482417,2.075974,...,1.739933,0.502638,1.298851,1.106010,0.515800,1.388353,0.709824,1.006039,1.048026,0.771120
2023.875000,2.143032,4.684096,3.500223,1.464903,1.737410,0.498751,1.100038,1.298444,1.168420,1.815787,...,1.900282,0.466283,1.158728,1.270223,0.864033,1.291394,1.375861,0.970561,1.037625,0.860709
